In [1]:
# Colab cell
try:
    import google.colab
    IN_COLAB = True
    !wget -q https://raw.githubusercontent.com/Competencia-de-Climate-Change/WeatherBench/master/src/ml_training.py
    !pip install -q -U zarr
    !pip install -q -U fsspec
    !pip install -q -U gcsfs
    !pip install -q -U xarray
except:
    IN_COLAB = False

In [2]:
import xarray as xr
import fsspec

ds_ = xr.open_zarr(fsspec.get_mapper('gcs://weather-bench/region_zarr/'), consolidated=True)

print("Tamaño en GB:", ds_.nbytes / 1e9)

Tamaño en GB: 19.324476644


In [3]:
import numpy as np

ds = ds_.assign(z_quad=ds_['z']**2, z_cub=ds_['z']**3, z_log=np.log(ds_['z']), t_quad=ds_['t']**2,
               t_cub=ds_['t']**3, t_log=np.log(ds_['t']))

print("Tamaño en GB:", ds.nbytes / 1e9)
ds

Tamaño en GB: 37.703622884


<xarray.Dataset>
Dimensions:    (lat: 12, level: 13, lon: 14, time: 350640)
Coordinates:
  * lat        (lat) float64 -70.31 -64.69 -59.06 ... -19.69 -14.06 -8.438
  * level      (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lon        (lon) float64 247.5 253.1 258.8 264.4 ... 303.8 309.4 315.0 320.6
  * time       (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Data variables:
    lat2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lon2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lsm        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    orography  (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    pv         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    q          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    r          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    slt        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    t          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    t2m        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    tcc        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    u10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    v10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    vo         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    z          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    z_quad     (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    z_cub      (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    z_log      (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    t_quad     (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    t_cub      (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    t_log      (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-08 20:51:24 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

# Non Linear Regression for $T850$ and $T2M$

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import pickle

from src.ml_training import *

# from dask_ml.wrappers import ParallelPostFit

def to_pickle(obj, fn):
    with open(fn, 'wb') as f:
        pickle.dump(obj, f)
def read_pickle(fn):
    with open(fn, 'rb') as f:
        return pickle.load(f)
    
results = 'results'
verbose = False

import os
if not os.path.exists(results):
    os.makedirs(results)

## Data Loading

In [6]:
z_500 = ds.z.sel(level=500).drop('level')
z_500_quad = ds.z_quad.sel(level=500).drop('level')
z_500_cub = ds.z_cub.sel(level=500).drop('level')
z_500_log = ds.z_log.sel(level=500).drop('level')

t_850 = ds.t.sel(level=850).drop('level')
t_850_quad = ds.t_quad.sel(level=850).drop('level')
t_850_cub = ds.t_cub.sel(level=850).drop('level')
t_850_log = ds.t_log.sel(level=850).drop('level')


t2m   = ds.t2m

data = xr.merge([z_500, z_500_quad, z_500_cub, z_500_log, t_850, t_850_quad, t_850_cub, t_850_log, t2m], compat='override')

print(data)


data_train = data.sel(time=slice('1979', '2016'))
data_test  = data.sel(time=slice('2017', '2018'))

z_test   = load_test_data('z', ds=ds)
z_quad_test   = load_test_data('z_quad', ds=ds)
z_cub_test   = load_test_data('z_cub', ds=ds)
z_log_test   = load_test_data('z_log', ds=ds)

t_test   = load_test_data('t', ds=ds)
t_quad_test   = load_test_data('t_quad', ds=ds)
t_cub_test   = load_test_data('t_cub', ds=ds)
t_log_test   = load_test_data('t_log', ds=ds)

t2m_test = load_test_data('t2m', ds=ds)

test_data = xr.merge([z_test, z_quad_test, z_cub_test, z_log_test, t_test,
                      t_quad_test, t_cub_test, t_log_test, t2m_test])

_, nlat, nlon = data_train.z.shape

nlat, nlon

<xarray.Dataset>
Dimensions:  (lat: 12, lon: 14, time: 350640)
Coordinates:
  * lat      (lat) float64 -70.31 -64.69 -59.06 -53.44 ... -19.69 -14.06 -8.438
  * lon      (lon) float64 247.5 253.1 258.8 264.4 ... 303.8 309.4 315.0 320.6
  * time     (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Data variables:
    z        (time, lat, lon) float32 dask.array<chunksize=(43830, 3, 4), meta=np.ndarray>
    z_quad   (time, lat, lon) float32 dask.array<chunksize=(43830, 3, 4), meta=np.ndarray>
    z_cub    (time, lat, lon) float32 dask.array<chunksize=(43830, 3, 4), meta=np.ndarray>
    z_log    (time, lat, lon) float32 dask.array<chunksize=(43830, 3, 4), meta=np.ndarray>
    t        (time, lat, lon) float32 dask.array<chunksize=(43830, 3, 4), meta=np.ndarray>
    t_quad   (time, lat, lon) float32 dask.array<chunksize=(43830, 3, 4), meta=np.ndarray>
    t_cub    (time, lat, lon) float32 dask.array<chunksize=(43830, 3, 4), meta=np.ndarray>
    t_log    (time, lat, lon) float32 dask

(12, 14)

In [7]:
# Compute normalization statistics
# Let's only take a sample of the time to speed this up.
data_mean = data_train.isel(time=slice(0, None, 10000)).mean().load()
data_std  = data_train.isel(time=slice(0, None, 10000)).std().load()


# Normalize datasets
data_train = (data_train - data_mean) / data_std
data_test  = (data_test - data_mean)  / data_std

### Define combinations of training data

In [15]:
# Inputs and Outputs
experiments = [
#     [['z'], ['z']],
#     [['t'], ['t']],
#     [['t', 't_quad', 't_log'], ['t']],
#     [['z', 't'], ['z', 't']],
#     [['t2m'], ['t2m']],
#     [['z', 't', 't2m'], ['z']],
    [['z', 'z_quad', 'z_cub', 'z_log', 't', 't_quad', 't_cub', 't_log', 't2m'], ['t']],
    [['z', 'z_quad', 'z_cub', 't', 't_quad', 't_cub', 't2m'], ['t']],
#     [['z', 'z_quad', 'z_cub', 'z_log', 't', 't_quad', 't_cub', 't_log', 't2m'], ['t2m']],
]

lead_time = 3*24 # Predict 3 days from current time
lead_time_str = '3d'

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import MultiTaskLasso, MultiTaskLassoCV
from sklearn.linear_model import MultiTaskElasticNet, MultiTaskElasticNetCV

In [17]:
preds  = {
    'lr'        : [],
#     'ridge'     : [],
#     'ridge_cv'  : [],
#     'lasso'     : [],
    'elasticnet': []
}

models = {
    'lr'        : [],
#     'ridge'     : [],
#     'ridge_cv'  : [],
#     'lasso'     : [],
    'elasticnet': []
}

train_mse = {
    'lr'        : [],
#     'ridge'     : [],
#     'ridge_cv'  : [],
#     'lasso'     : [],
    'elasticnet': []
}

test_mse = {
    'lr'        : [],
#     'ridge'     : [],
#     'ridge_cv'  : [],
#     'lasso'     : [],
    'elasticnet': []
}

weighted_mse = {
    'lr'        : [],
#     'ridge'     : [],
#     'ridge_cv'  : [],
#     'lasso'     : [],
    'elasticnet': []
}

model_args = [
    [LinearRegression,    'lr', {}], 
#     [Ridge,               'ridge', {}], 
#     [RidgeCV,             'ridge_cv', {'alphas': [0.001, 0.01, 1., 10., 50.]}],
#     [MultiTaskLasso,      'lasso', {'tol' : 0.001, 'selection' : 'random'}],
    [MultiTaskElasticNet, 'elasticnet', {'tol' : 0.001, 'selection' : 'random'}]
]

## Run Experiments

In [18]:
for model in model_args:
    model_fun, model_str, extra_args = model
    
    print('Running:', model_str)
    for idx, (i, o) in enumerate(experiments):
        print(f'{idx}: Input variables = {i}; output variables = {o}')
        
        X_train, y_train, X_test, y_test, time = create_X_y_time(input_vars=i, 
                                                                 output_vars=o, 
                                                                 data=[data_train, data_test], 
                                                                 lead_time=lead_time, 
                                                                 data_subsample=5)
        
        data_needed_train = [X_train, y_train, X_test, y_test, nlat, nlon]
        
        predictions, model_res, mse_train, mse_test = train_regression(model=model_fun,
                                                                       data=data_needed_train,
                                                                       num_outputs=len(o),
                                                                       extra_args=extra_args,
                                                                       verbose=False)
        
        predictions = unnormalize_preds(preds=predictions, 
                                        output_vars=o, 
                                        valid_time=time, 
                                        lat_lon=[data_train.lat, data_train.lon],
                                        data_std=data_std, 
                                        data_mean=data_mean)

        preds[model_str].append(predictions)
        models[model_str].append(model_res)

        # Compute Weighted RMSE: RMSE with a weight that takes into account grid location
        r = compute_weighted_rmse(predictions, test_data).compute()
        if verbose:
            print('; '.join([f'{v} = {r[v].values}' for v in r]) + '\n')
        
        train_mse[model_str].append(mse_train)
        test_mse[model_str].append(mse_test)
        weighted_mse[model_str].append(r)

        # Save models
        preds_nc_str  = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.nc'
        model_res_str = f'{results}/{model_str}_{lead_time_str}_{"_".join(i)}_{"_".join(o)}.pkl'
        
        predictions.to_netcdf(preds_nc_str)
        to_pickle(model_res, model_res_str)
    print('----------------------------------------------------------------------------')

Running: lr
0: Input variables = ['z', 'z_quad', 'z_cub', 'z_log', 't', 't_quad', 't_cub', 't_log', 't2m']; output variables = ['t']
1: Input variables = ['z', 'z_quad', 'z_cub', 't', 't_quad', 't_cub', 't2m']; output variables = ['t']
----------------------------------------------------------------------------
Running: elasticnet
0: Input variables = ['z', 'z_quad', 'z_cub', 'z_log', 't', 't_quad', 't_cub', 't_log', 't2m']; output variables = ['t']
1: Input variables = ['z', 'z_quad', 'z_cub', 't', 't_quad', 't_cub', 't2m']; output variables = ['t']
----------------------------------------------------------------------------


In [ ]:
# def apply_polinomial(X, deg=3):
#     n = X.shape[0]
#     Phi = np.ones(n)
#     for i in range(1, deg+1):
#         Phi = np.vstack([Phi, X**i])
#     return Phi.T

# def apply_coseno(X, deg=1, T=2):
#     """
#     X: 1d observations
#     d: degree
#     T: period
#     """
#     phi_x = np.ones((len(x), deg+1)) 
#     for d in range(1, deg + 1):
#         phi_x[:, d] = np.cos(d * 2 * np.pi / deg / T * x)
#     return phi_x

# for count, degree in enumerate([3, 4, 5]):
#     model = make_pipeline(PolynomialFeatures(degree), Ridge())
#     model.fit(X, y)
#     y_plot = model.predict(X_plot)
#     plt.plot(x_plot, y_plot, color=colors[count], linewidth=lw,
#              label="degree %d" % degree)

In [19]:
import pandas as pd

results_df = pd.DataFrame(columns=['model', 'input', 'output',  'train_mse', 'test_mse', 'weighted_mse'])

block_rows = []
for key in models.keys():
    print(key)
    # repeat key as many time as needed to have a long df
    first_three_cols = np.hstack((np.array([key] * len(experiments)).reshape(len(experiments),1), 
                                  np.array(experiments, dtype='object')))
    block_rows.append(first_three_cols)
    
block_rows = np.array(block_rows).reshape(len(models.keys()) * len(experiments), 3) 

results_df['model'] = block_rows[:, 0]
results_df['input'] = block_rows[:, 1]
results_df['output'] = block_rows[:, 2]


train_mse_col = []
for model, ds_list in train_mse.items():
    train_mse_col += ds_list

test_mse_col = []
for model, ds_list in test_mse.items():
    test_mse_col += ds_list
    

weighted_mse_col = []
for model, ds_list in weighted_mse.items():
    print(model)
    model_metrics = []
    for ds in ds_list:
        exp_metrics = []
        for var in list(ds.variables):
            exp_metrics.append(round(float(ds[var].values), 4))

        model_metrics.append(exp_metrics)
    weighted_mse_col += model_metrics

    
print('\nNumber of rows equals number of metrics? ', results_df.shape[0] == len(train_mse_col))
print(results_df.shape[0], len(train_mse_col))


results_df['train_mse'] = train_mse_col
results_df['test_mse'] = test_mse_col

results_df['weighted_mse'] = weighted_mse_col
results_df['weighted_mse'] = results_df['weighted_mse'].apply(lambda x : x[0])
results_df

lr
lr

Number of rows equals number of metrics?  True
3 3


,model,input,output,train_mse,test_mse,weighted_mse
0,lr,"[t, t_quad, t_log]",[t],0.104679,0.115668,3.3020
1,lr,"[z, z_quad, z_cub, z_log, t, t_quad, t_cub, t_...",[t],0.096948,0.117585,3.3304
2,lr,"[z, z_quad, z_cub, z_log, t, t_quad, t_cub, t_...",[t2m],0.035425,0.040215,2.0000


In [ ]:
import seaborn as sns
sns.set_style('darkgrid')

def get_indexes(get_var):
    idxs = []
    for idx, out in results_df['output'].items():
        for var in out:
            var = str(var).lower()

            if get_var == var:
                idxs.append(idx)
    return idxs

idxs = get_indexes('t2m')
t2m_out = results_df[results_df['output'].index.isin(idxs)]

f, axs = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))

sns.lineplot(data=t2m_out, x='model', y='train_mse', ax=axs[0])
sns.lineplot(data=t2m_out, x='model', y='test_mse',  ax=axs[1])
sns.lineplot(data=t2m_out, x='model', y='weighted_mse', ax=axs[2])
f.show()


idxs = get_indexes('t')
t_out = results_df[results_df['output'].index.isin(idxs)]

f, axs = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))

sns.lineplot(data=t_out, x='model', y='train_mse', ax=axs[0])
sns.lineplot(data=t_out, x='model', y='test_mse',  ax=axs[1])
sns.lineplot(data=t_out, x='model', y='weighted_mse', ax=axs[2])

f.show()


idxs = get_indexes('z')
z_out = results_df[results_df['output'].index.isin(idxs)]

f, axs = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))

sns.lineplot(data=z_out, x='model', y='train_mse', ax=axs[0])
sns.lineplot(data=t_out, x='model', y='test_mse',  ax=axs[1])
sns.lineplot(data=t_out, x='model', y='weighted_mse', ax=axs[2])
f.show()

## Iterative Linear Regression

In [ ]:
# def create_iterative_lr(state, model, lead_time_h=6, max_lead_time_h=5*24):
    
#     max_fc_steps = max_lead_time_h // lead_time_h

#     preds_z500, preds_t850 = [], []

#     for fc_step in tqdm(range(max_fc_steps)):
#         # predict next state and update current with next
#         state = model.predict(state)
        
#         # Unnormalize
#         fc_z500 = state[:, :nlat*nlon].copy() * data_std.z.values + data_mean.z.values
#         fc_t850 = state[:, nlat*nlon:].copy() * data_std.t.values + data_mean.t.values
        
#         # Reshape
#         fc_z500 = fc_z500.reshape((-1, nlat, nlon))
#         fc_t850 = fc_t850.reshape((-1, nlat, nlon))
        
#         preds_z500.append(fc_z500)
#         preds_t850.append(fc_t850)

#     return [xr.DataArray(
#         np.array(fcs), 
#         dims=['lead_time', 'time', 'lat', 'lon'],
#         coords={
#             'lead_time': np.arange(lead_time_h, max_lead_time_h + lead_time_h, lead_time_h),
#             'time': z_test.time,
#             'lat': z_test.lat,
#             'lon': z_test.lon
#         }
#     ) for fcs in [preds_z500, preds_t850]]

In [ ]:
# p, m = train_regression(LinearRegression, 
#                         6, 
#                         input_vars=['z', 't'], 
#                         output_vars=['z', 't'])

In [ ]:
# state = np.concatenate([data_test.z.values.reshape(-1, nlat*nlon), 
#                         data_test.t.values.reshape(-1, nlat*nlon)], 1)

In [ ]:
# fc_z500_6h_iter, fc_t850_6h_iter = create_iterative_lr(state, m)

In [ ]:
# fc_z500_6h_iter